In [10]:
# !pip install requests

In [11]:
import requests
import re
import time
import json

def limpar_cnpj(cnpj):
	"""Remove todos os caracteres não numéricos de um CNPJ."""
	return re.sub(r'[^0-9]', '', cnpj)

def consultar_cnpjs(lista_cnpjs):
	"""Consulta uma lista de CNPJs na BrasilAPI."""
	
	base_url = "https://brasilapi.com.br/api/cnpj/v1/"
	
	resultados = {}
	
	for cnpj_formatado in lista_cnpjs:
		
		cnpj_limpo = limpar_cnpj(cnpj_formatado)
		
		url_consulta = f"{base_url}{cnpj_limpo}"
		
		print(f"\n*** CNPJ: {cnpj_formatado}")
		
		try:
			
			# Requisição GET ---- 'timeout' para não ficar esperando pra sempre
			response = requests.get(url_consulta, timeout=10)
			
			# * Código 200 - Consulta bem-sucedida
			if response.status_code == 200:
				dados_empresa = response.json()
				resultados[cnpj_formatado] = dados_empresa
				
				print(resultados[cnpj_formatado])
				
			# Erros esperados da API
            # 404 - CNPJ não encontrado
			elif response.status_code == 404:
				print(f"Erro: CNPJ '{cnpj_formatado}' não encontrado.")
				resultados[cnpj_formatado] = {"erro": "CNPJ não encontrado"}
			# Outros erros
			else:
				print(f"Erro ao consultar: Status Code {response.status_code}")
				resultados[cnpj_formatado] = {"erro": f"Status {response.status_code}"}
		
		# Erros de conexão (sem internet, timeout, etc.)
		except requests.exceptions.RequestException as e:

			print(f"Erro de conexão: {e}")
			resultados[cnpj_formatado] = {"erro": f"Erro de conexão: {e}"}
		
		print("\n" + "#" * 40 + "\n")
		
		# Uma pequena pausa é uma boa prática para não sobrecarregar a API
		time.sleep(1) 
		
	return resultados

# Lista de CNPJs para consulta
meus_cnpjs = [
	"45.954.282/0001-02",   # CNPJ da própria Principia Educação
	"00.000.000/0001-91",	# Banco do Brasil (Exemplo Matriz)
	"19.131.243/0001-97", 	# Exemplo de empresa
	"99999999000191",		# Exemplo de CNPJ inválido
]

dados_das_empresas = consultar_cnpjs(meus_cnpjs)

# Tranformar em JSON
dados_em_json = json.dumps(dados_das_empresas, indent=4, ensure_ascii=False)
with open("data/BrasilAPI_dados_empresas.json", "w", encoding="utf-8") as arquivo:
	arquivo.write(dados_em_json)



*** CNPJ: 45.954.282/0001-02
{'uf': 'SP', 'cep': '04548903', 'qsa': [{'pais': 'ESTADOS UNIDOS', 'nome_socio': 'PRINCIPIA EDUCATION TECHNOLOGY LLC', 'codigo_pais': 249, 'faixa_etaria': 'Não se aplica', 'cnpj_cpf_do_socio': '48249302000124', 'qualificacao_socio': 'Sócio Pessoa Jurídica Domiciliado no Exterior', 'codigo_faixa_etaria': 0, 'data_entrada_sociedade': '2022-11-17', 'identificador_de_socio': 1, 'cpf_representante_legal': '***658924**', 'nome_representante_legal': 'NEWTON MAIA SALOMAO ALVES', 'codigo_qualificacao_socio': 37, 'qualificacao_representante_legal': 'Procurador', 'codigo_qualificacao_representante_legal': 17}, {'pais': None, 'nome_socio': 'NEWTON MAIA SALOMAO ALVES', 'codigo_pais': None, 'faixa_etaria': 'Entre 41 a 50 anos', 'cnpj_cpf_do_socio': '***658924**', 'qualificacao_socio': 'Administrador', 'codigo_faixa_etaria': 5, 'data_entrada_sociedade': '2022-04-07', 'identificador_de_socio': 2, 'cpf_representante_legal': '***000000**', 'nome_representante_legal': '', 'c